In [ ]:
%matplotlib inline

# 텐서(Tensors)

텐서는 배열(arrays) 및 행렬(matrix)과 매우 유사한 전문화된 데이터 구조입니다.
파이토치에서는 텐서를 사용하여 모델의 입력 및 출력뿐만 아니라 모델의 파라미터를 인코딩합니다.

텐서는 [Numpy](https://numpy.org/)의 ndarrays와 유사하지만 텐서는 GPU 또는 다른 하드웨어 가속기에서 실행할 수 있다. 실제로 텐서와 NumPy 배열은 동일한 기본 메모리를 공유할 수 있으므로 데이터를 복사할 필요가 없다(`bridge-to-np-label` 참조). 텐서는 자동 미분에 최적화되어 있습니다(이 후 Autograd 부분에서 자세히 살펴볼 예정입니다.). 당신이 ndarrays에 익숙하다면 Tensor API를 바로 사용할 수 있습니다. 따라오세요!

환경 조성하는 것부터 시작합시다.


In [ ]:
import torch
import numpy as np

# 텐서 생성

텐서는 다양한 방법으로 생성할 수 있습니다. 다음 예를 살펴봅시다:


## 데이터로부터 직접 생성

텐서는 데이터에서 직접 만들 수 있습니다. 데이터의 유형은 자동으로 유추됩니다.



In [ ]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

## NumPy 배열에서부터 생성

Tensors can be created from NumPy arrays (and vice versa - see `bridge-to-np-label`).



In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

## 다른 텐서로부터 생성:

새로운 텐서는 명시적으로 재정의되지 않는 한 텐서의 속성(모양, 데이터 타입)을 유지한다.



In [ ]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

## 무작위 또는 상수 값을 사용하여 생성:

``모양(Shape)``은 텐서 차원의 튜플(tuple)입니다. 아래 함수에서 출력 텐서의 차원을 결정합니다.



In [ ]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

# 텐서의 특성

텐서의 특성은 모양, 데이터 타입 및 텐서가 저장되는 장치를 설명합니다.



In [ ]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

# 텐서 연산

산술, 선형 대수, 행렬 조작(전치(transposing), 인덱싱, 슬라이스), 샘플링 등 100개 이상의 텐서 연산이 있으며
[여기](https://pytorch.org/docs/stable/torch.html)에 더욱 자세하게 기술되어 있다.

이러한 각 작업은 GPU(일반적으로 CPU보다 빠른 속도)에서 실행할 수 있습니다.

기본적으로 텐서는 CPU에서 생성됩니다. 우리는 GPU의 가용성을 확인한 후, `.to` 메서드를 사용하여 
텐서를 GPU로 명시적으로 이동시켜야 한다. 장치 간에 대규모의 텐서를 복사하는 것은 시간과 메모리 측면에서
비용이 많이 들 수 있다는 점을 명심하십시오!


In [ ]:
# We move our tensor to the GPU if available
if torch.cuda.is_available():
  tensor = tensor.to('cuda')

목록에서 몇 가지 작업을 시도해 보십시오. NumPy API를 잘 알고 있으면 Tensor API를 쉽게 사용할 수 있습니다.

## 표준 numpy 방식의 인덱싱과 슬라이싱:

In [ ]:
tensor = torch.ones(4, 4)
print('First row: ',tensor[0])
print('First column: ', tensor[:, 0])
print('Last column:', tensor[..., -1])
tensor[:,1] = 0
print(tensor)

## 텐서 결합

`torch.cat`을 사용하여 주어진 차원을 따라 일련의 텐서를 결합할 수 있습니다.
``torch.cat``과 약간 다른 텐서 결합 메서드인 [torch.stack](https://pytorch.org/docs/stable/generated/torch.stack.html)을 참조하십시오,



In [ ]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

## 산술 연산



In [ ]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(tensor)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

## 단일 요소 텐서

예를 들어 텐서의 모든 값을 하나의 값으로 집계하여 one-element 텐서가 있는 경우, `item()`을 사용하여 이를 Python numerial 값으로 변환할 수 있다.


In [ ]:
agg = tensor.sum()
agg_item = agg.item()  
print(agg_item, type(agg_item))

## 인플레이스(In-place) 연산

결과를 피연산자에 저장하는 연산을 In-place라고 합니다. 그들은 ``_`` 접미사에 의해 표시됩니다.
예를 들어 : ``x.copy_(y)``, ``x.t_()``는 ``x``가 바뀌게 될 것 입니다.


> **Note:** 인플레이스 연산은 약간의 메모리를 절약하지만, 기록이 즉시 손실되기 때문에 파생 항목을 계산할 때 문제가 될 수 있어 사용하지 않는 것이 좋습니다.

In [ ]:
print(tensor, "\n")
tensor.add_(5)
print(tensor)

## Bridge with NumPy

CPU 및 NumPy Array의 Tensor는 기본 메모리 위치를 공유 할 수 있으며 하나를 변경하면 나머지도 같이 변경됩니다.

### Tensor to NumPy array

In [ ]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

Tensor의 변경은 NumPy Array에 반영됩니다.

In [ ]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

### NumPy array to Tensor

In [ ]:
n = np.ones(5)
t = torch.from_numpy(n)

NumPy Array의 변경은 Tensor에 반영됩니다.

In [ ]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")